In [1]:
%pip install pandas
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy>=1.18.5 in c:\anaconda3-202205\lib\site-packages (from pandas) (1.21.5)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import io

import os

import sys

import string

import numpy as np

import pandas as pd

from tensorflow import keras

from __future__ import print_function

from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping

from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding


translator = str.maketrans('', '', string.punctuation)

df = pd.read_csv("./lyrics-data.csv")

df.head()





#pdf = pd.read_csv('./data/PoetryFoundationData.csv',quotechar='"')

#pdf.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [ ]:
def split_text(x):

   text = x['Lyric']

    
   sections = str(text).split()

   keys = {'Verse 1': np.nan,'Verse 2':np.nan,'Verse 3':np.nan,'Verse 4':np.nan, 'Chorus':np.nan}

   lyrics = str()

   single_text = []

   res = {}

   for s in sections:

       key = s[s.find('[') + 1:s.find(']')].strip()

       if ':' in key:

           key = key[:key.find(':')]
          

       if key in keys:

           single_text += [x.lower().replace('(','').replace(')','').translate(translator) for x in s[s.find(']')+1:].split('\\n') if len(x) > 1]
          

       res['single_text'] =  ' \n '.join(single_text)

   return pd.Series(res)



In [48]:
df = pd.read_csv("./lyrics-data.csv")
#df = df.join( df.apply(split_text, axis=1))
#sum_df = pd.DataFrame( df['single_text'] )



sum_df.dropna(inplace=True)
df.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [50]:
sum_df = pd.DataFrame( df['single_text'] )
sum_df

KeyError: 'single_text'

In [45]:
text_as_list = []

frequencies = {}

uncommon_words = set()

MIN_FREQUENCY = 2

MIN_SEQ = 5

BATCH_SIZE =  32

print("Starting")


def extract_text(text):

   global text_as_list

   text_as_list += [w for w in text.split(' ') if w.strip() != '' and w != '\n' and w != 'chorus']



sum_df['single_text'].apply( extract_text )

print('Total words: ', len(text_as_list))

print(text_as_list)
for w in text_as_list:

   frequencies[w] = frequencies.get(w, 0) + 1
  

uncommon_words = set([key for key in frequencies.keys() if frequencies[key] < MIN_FREQUENCY])

words = sorted(set([key for key in frequencies.keys() if frequencies[key] >= MIN_FREQUENCY]))


num_words = len(words)

word_indices = dict((w, i) for i, w in enumerate(words))

indices_word = dict((i, w) for i, w in enumerate(words))

print('Words with less than {} appearances: {}'.format( MIN_FREQUENCY, len(uncommon_words)))

print('Words with more than {} appearances: {}'.format( MIN_FREQUENCY, len(words)))


valid_seqs = []

end_seq_words = []

for i in range(len(text_as_list) - MIN_SEQ ):

   end_slice = i + MIN_SEQ + 1

   if len( set(text_as_list[i:end_slice]).intersection(uncommon_words) ) == 0:

       valid_seqs.append(text_as_list[i: i + MIN_SEQ])

       end_seq_words.append(text_as_list[i + MIN_SEQ])
      

print('Valid sequences of size {}: {}'.format(MIN_SEQ, len(valid_seqs)))


X_train, X_test, y_train, y_test = train_test_split(valid_seqs, end_seq_words, test_size=0.02, random_state=42)

print(X_train[2:5])

Starting
Total words:  193
['2x', '2x', '2x', '2x', 'chorus50', 'chorus1', 'chorus2', 'chorus3', 'chorus4', 'x2', 'chorus2x', 'chorus4x', '2x', 'x4', 'chorus2x', 'chorus2x', 'chorus2x', 'chorusif', 'x2', 'chorusbaby', 'chorusjazze', 'x2', 'x2', 'x2', '2x', 'chorus2', 'how', 'how', 'repeated', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'x2', 'chorus2x', 'chorusbut', 'chorusyoyoyoyon', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'chorus…', 'chorusthe', 'chorusthe', 'chorusthe', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'chorusand', 'choruswomen', 'choruswomen', 'chorus2x', 'chorusbobby', 'ja', 'chorustweet', '2x', 'flo', 'flo', 'flo', 'flo', 'x2', 'nate', 'nate', 'nate', 'chorusjd', '50', '50', '50', '50', '50', 'chorus50', '50', 'x4', '2xti', 'choruskanye', 'choruswyclef', 'choruswyclef', 'eminem', 'eminem', 'eminem', 'eminem', 'chorusall', 'chorusall', 'chorusall', 'chorus4x', 'chorus…', 'chorus…

In [6]:
# Data generator for fit and evaluate

def generator(sentence_list, next_word_list, batch_size):

   index = 0

   while True:

       x = np.zeros((batch_size, MIN_SEQ), dtype=np.int32)

       y = np.zeros((batch_size), dtype=np.int32)

       for i in range(batch_size):

           for t, w in enumerate(sentence_list[index % len(sentence_list)]):

               x[i, t] = word_indices[w]

           y[i] = word_indices[next_word_list[index % len(sentence_list)]]

           index = index + 1

       yield x, y


# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py

def sample(preds, temperature=1.0):

   # helper function to sample an index from a probability array

   preds = np.asarray(preds).astype('float64')

   preds = np.log(preds) / temperature

   exp_preds = np.exp(preds)

   preds = exp_preds / np.sum(exp_preds)

   probas = np.random.multinomial(1, preds, 1)

   return np.argmax(probas)


def on_epoch_end(epoch, logs):

   # Function invoked at end of each epoch. Prints generated text.

   examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)


   # Randomly pick a seed sequence

   seed_index = np.random.randint(len(X_train+X_test))

   seed = (X_train+X_test)[seed_index]


   for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:

       sentence = seed

       examples_file.write('----- Diversity:' + str(diversity) + '\n')

       examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')

       examples_file.write(' '.join(sentence))


       for i in range(50):

           x_pred = np.zeros((1, MIN_SEQ))

           for t, word in enumerate(sentence):

               x_pred[0, t] = word_indices[word]


           preds = model.predict(x_pred, verbose=0)[0]

           next_index = sample(preds, diversity)

           next_word = indices_word[next_index]


           sentence = sentence[1:]

           sentence.append(next_word)


           examples_file.write(" "+next_word)

       examples_file.write('\n')

   examples_file.write('='*80 + '\n')

   examples_file.flush()

In [7]:
def get_model():

   print('Build model...')

   model = Sequential()

   model.add(Embedding(input_dim=len(words), output_dim=1024))

   model.add(Bidirectional(LSTM(128)))

   model.add(Dense(len(words)))

   model.add(Activation('softmax'))

   return model

In [25]:
model = get_model()

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

#file_path = "./checkpoints\LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-acc{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}(len(words), MIN_SEQ, MIN_FREQUENCY)"

file_path = "C:/Users/KOLLENBE/Desktop/models/{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-acc{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}(len(words), MIN_SEQ, MIN_FREQUENCY)"
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)

callbacks_list = [checkpoint, print_callback, early_stopping]


examples_file = open('examples.txt', "w")

model.fit(generator(X_train, y_train, BATCH_SIZE),

                   steps_per_epoch=int(len(valid_seqs)/BATCH_SIZE) + 1,

                   epochs=20,

                   callbacks=callbacks_list,

                   validation_data=generator(X_test, y_train, BATCH_SIZE),

                   validation_steps=int(len(y_train)/BATCH_SIZE) + 1)

Build model...
Epoch 1/20
441/443 [============================>.] - ETA: 0s - loss: 0.5258 - accuracy: 0.7284

INFO:tensorflow:Assets written to: C:/Users/KOLLENBE/Desktop/models\001-words%d-sequence%d-minfreq%d-loss0.5259-acc0.7285-val_loss0.9796-val_acc0.6091(len(words), MIN_SEQ, MIN_FREQUENCY)\assets


INFO:tensorflow:Assets written to: C:/Users/KOLLENBE/Desktop/models\001-words%d-sequence%d-minfreq%d-loss0.5259-acc0.7285-val_loss0.9796-val_acc0.6091(len(words), MIN_SEQ, MIN_FREQUENCY)\assets


443/443 [==============================] - 42s 88ms/step - loss: 0.5259 - accuracy: 0.7285 - val_loss: 0.9796 - val_accuracy: 0.6091
Epoch 2/20
443/443 [==============================] - 20s 45ms/step - loss: 0.4859 - accuracy: 0.7438 - val_loss: 1.0182 - val_accuracy: 0.6091
Epoch 3/20
443/443 [==============================] - 21s 46ms/step - loss: 0.4807 - accuracy: 0.7453 - val_loss: 1.0445 - val_accuracy: 0.5669
Epoch 4/20
443/443 [==============================] - 19s 44ms/step - loss: 0.4776 - accuracy: 0.7475 - val_loss: 1.0623 - val_accuracy: 0.6091
Epoch 5/20
443/443 [==============================] - 22s 50ms/step - loss: 0.4766 - accuracy: 0.7463 - val_loss: 1.0947 - val_accuracy: 0.5634
Epoch 6/20
441/443 [============================>.] - ETA: 0s - loss: 0.4749 - accuracy: 0.7484

INFO:tensorflow:Assets written to: C:/Users/KOLLENBE/Desktop/models\006-words%d-sequence%d-minfreq%d-loss0.4746-acc0.7485-val_loss1.1108-val_acc0.6302(len(words), MIN_SEQ, MIN_FREQUENCY)\assets


...

443/443 [==============================] - 40s 90ms/step - loss: 0.4746 - accuracy: 0.7485 - val_loss: 1.1108 - val_accuracy: 0.6302
Epoch 7/20
442/443 [============================>.] - ETA: 0s - loss: 0.4742 - accuracy: 0.7486

INFO:tensorflow:Assets written to: C:/Users/KOLLENBE/Desktop/models\007-words%d-sequence%d-minfreq%d-loss0.4740-acc0.7487-val_loss1.1257-val_acc0.6796(len(words), MIN_SEQ, MIN_FREQUENCY)\assets


INFO:tensorflow:Assets written to: C:/Users/KOLLENBE/Desktop/models\007-words%d-sequence%d-minfreq%d-loss0.4740-acc0.7487-val_loss1.1257-val_acc0.6796(len(words), MIN_SEQ, MIN_FREQUENCY)\assets


443/443 [==============================] - 38s 85ms/step - loss: 0.4740 - accuracy: 0.7487 - val_loss: 1.1257 - val_accuracy: 0.6796
Epoch 8/20
443/443 [==============================] - 20s 46ms/step - loss: 0.4741 - accuracy: 0.7492 - val_loss: 1.1486 - val_accuracy: 0.6303
Epoch 9/20
234/443 [==============>...............] - ETA: 4s - loss: 0.4709 - accuracy: 0.7479

KeyboardInterrupt: 